In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel
from transformers import get_scheduler
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split

c:\Users\timok\Documents\HSKA\Info_Master\2.Semester\AI_Labor\AI_Project\aiProjectVenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class EssayDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=512):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        targets = torch.tensor(self.targets[idx], dtype=torch.float)

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'targets': targets
        }


In [3]:
class BertRegressionModel(BertPreTrainedModel):
    def __init__(self, config, num_outputs=6):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.regressor = nn.Linear(config.hidden_size, num_outputs)

    def forward(self, input_ids, attention_mask, targets=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls_output)
        preds = self.regressor(x)
        return preds


In [4]:
df = pd.read_csv("./data/train.csv")
texts = df["full_text"].tolist()
targets = df[["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]].values

train_texts, val_texts, train_targets, val_targets = train_test_split(texts, targets, test_size=0.1)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = EssayDataset(train_texts, train_targets, tokenizer)
val_dataset = EssayDataset(val_texts, val_targets, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertRegressionModel.from_pretrained("bert-base-uncased", num_outputs=6)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.MSELoss()
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1}: loss = {loss.item():.4f}")


Some weights of BertRegressionModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
import pandas as pd

test_df = pd.read_csv("test.csv")  # Passe den Pfad an
test_texts = test_df["full_text"].tolist()


In [ ]:
test_dataset = EssayDataset(test_texts, targets=[[0]*6]*len(test_texts), tokenizer=tokenizer)  # Targets dummy
test_loader = DataLoader(test_dataset, batch_size=8)


In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions.append(outputs.cpu().numpy())

import numpy as np
predictions = np.vstack(predictions)


In [ ]:
# Spaltennamen wie in Training
columns = ["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]

# Als DataFrame speichern
pred_df = pd.DataFrame(predictions, columns=columns)

# Falls du den text_id wieder hinzufügen willst:
if "text_id" in test_df.columns:
    pred_df.insert(0, "text_id", test_df["text_id"])

# CSV exportieren
pred_df.to_csv("predictions.csv", index=False)


In [ ]:
def round_half(x):
    return round(x * 2) / 2

pred_df[columns] = pred_df[columns].applymap(round_half)

display(pred_df)